In [44]:
# Importing the libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as stats
import json
import copy

In [45]:
with open('../models/config.json', 'r') as f:
    config = json.load(f)

preprocess = config['preprocessing']

In [46]:
# Output Path
OUTPUT_PATH =preprocess['output_path']

# Input Path
INPUT_PATH = preprocess['input_path']

In [47]:
# Load the dataset
dataset = pd.read_csv(INPUT_PATH + '240624_validation_data.csv')


In [48]:
# Select the date for which you want to extract the data
date = '2023-07-10'

# Select the periode for which you want to extract the data
selected_period = 0 # 0 for day, 1 for week, 2 for month

# Number of scenarios to generate
num_scenarios = 1000
desired_num_scenarios = 10


In [49]:
# Extract the time, forecasted demand and actual demand
time = dataset['time']
time = pd.to_datetime(time, utc=True)
forecast_demand = dataset['predicted heat']
actual_demand = dataset['delivered heat']

# Create a dataframe with the necessary columns
df_data = pd.DataFrame({'time' : time, 'actual_demand': actual_demand, 'forecast_demand': forecast_demand})
df_data['hour']=time.dt.hour

# Calculate the error and absolute error
df_data['error']= df_data['actual_demand'] - df_data['forecast_demand']

# Calculate the mean of the error for whole dataset
mu_all = df_data['error'].mean()
sigma_all = df_data['error'].std()
print('Mean of of the error of each hour:', mu_all)
print('Standard deviation of the mean of the error of each hour:', sigma_all)

df_data


Mean of of the error of each hour: -8.39020298257356
Standard deviation of the mean of the error of each hour: 31.42172005784749


,time,actual_demand,forecast_demand,hour,error
0,2023-03-02 16:00:00+00:00,254.990005,255.590651,16,-0.600646
1,2023-03-02 17:00:00+00:00,187.787503,237.501176,17,-49.713673
2,2023-03-02 18:00:00+00:00,230.629171,264.220971,18,-33.591800
3,2023-03-02 19:00:00+00:00,233.329169,236.491860,19,-3.162691
4,2023-03-02 20:00:00+00:00,222.775003,216.179837,20,6.595167
...,...,...,...,...,...
2996,2023-07-12 19:00:00+00:00,55.804167,51.912949,19,3.891219
2997,2023-07-12 20:00:00+00:00,14.191667,37.334409,20,-23.142742
2998,2023-07-12 21:00:00+00:00,22.533334,37.278034,21,-14.744700
2999,2023-07-12 22:00:00+00:00,15.287500,37.703201,22,-22.415700


In [50]:
# Calculate the mean and standard deviation of the error of each hour of a day. 
error = df_data.groupby('hour')['error'].agg(['mean', 'std']).reset_index()
error  = error.rename(columns={'mean': 'mu', 'std': 'sigma'})

# Save the error data to a dictionary
error_dict = error.set_index('hour').to_dict(orient='index')

# Print the error dictionary
error_dict



{0: {'mu': -12.731658021190215, 'sigma': 25.018753157221795},
 1: {'mu': -8.717701939869812, 'sigma': 25.787822594592562},
 2: {'mu': 38.62809145641247, 'sigma': 40.06221098717317},
 3: {'mu': -11.219355528775246, 'sigma': 35.89610812413864},
 4: {'mu': 4.6619634940865975, 'sigma': 29.89150310430875},
 5: {'mu': -3.1503448513536596, 'sigma': 30.62503489088334},
 6: {'mu': -11.665524616604312, 'sigma': 28.687483429676817},
 7: {'mu': -8.965185204385374, 'sigma': 30.89163216771525},
 8: {'mu': -4.254730131452734, 'sigma': 29.0944311173442},
 9: {'mu': -9.849400683302798, 'sigma': 33.982800569041345},
 10: {'mu': -16.93716965553407, 'sigma': 31.972997351520558},
 11: {'mu': -11.808679656691835, 'sigma': 27.301664271959485},
 12: {'mu': -14.417149987864374, 'sigma': 29.132160670687032},
 13: {'mu': -11.036739802937781, 'sigma': 27.055038468354763},
 14: {'mu': -14.235238982654698, 'sigma': 29.627671262886945},
 15: {'mu': -12.303575238253774, 'sigma': 33.18363957415679},
 16: {'mu': -7.271

In [51]:
# Convert selected_date to datetime with UTC timezone
selected_date = date
selected_date = pd.to_datetime(selected_date, utc=True)

# Determine the start and end dates based on the selected period
if selected_period == 0:  # Day
    start_date = selected_date.normalize()
    end_date = start_date + pd.Timedelta(hours=23)
elif selected_period == 1:  # Week
    start_date = selected_date.normalize()
    end_date = start_date + pd.Timedelta(days=6, hours=23)
elif selected_period == 2:  # Month
    start_date = selected_date.normalize()
    end_date = (start_date + pd.DateOffset(months=1)) - pd.Timedelta(hours=1)
else:
    raise ValueError("selected_period must be 0 (day), 1 (week), or 2 (month)")

# Filter data for the selected period
period_data = df_data[(df_data['time'] >= start_date) & (df_data['time'] <= end_date)].reset_index(drop=True)
period_data = period_data[['time', 'hour', 'actual_demand', 'forecast_demand', 'error']]

# Generate expected timestamps
expected_time = pd.date_range(
    start=start_date,
    end=end_date,
    freq='H',
    tz='UTC'
)

# Check for missing timestamps
missing_time = expected_time.difference(period_data['time'])

print('Expected number of data points:', len(expected_time))
print('Actual number of data points:', len(period_data))

if missing_time.empty:
    print('The selected period has all the expected data.')
else:
    print(f'The selected period is missing {len(missing_time)} data points.')
    print('Missing timestamps:')
    print(missing_time)

# Display the filtered data
period_data

Expected number of data points: 24
Actual number of data points: 24
The selected period has all the expected data.


C:\Users\paul\AppData\Local\Temp\ipykernel_19876\1721325174.py:23: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  expected_time = pd.date_range(


,time,hour,actual_demand,forecast_demand,error
0,2023-07-10 00:00:00+00:00,0,19.491667,37.182262,-17.690595
1,2023-07-10 01:00:00+00:00,1,42.829167,44.471830,-1.642663
2,2023-07-10 02:00:00+00:00,2,144.383336,87.021062,57.362274
3,2023-07-10 03:00:00+00:00,3,42.000000,59.673830,-17.673829
4,2023-07-10 04:00:00+00:00,4,70.545834,63.712583,6.833251
5,2023-07-10 05:00:00+00:00,5,29.120834,61.861487,-32.740654
6,2023-07-10 06:00:00+00:00,6,45.058334,60.155867,-15.097533
7,2023-07-10 07:00:00+00:00,7,50.479167,55.959689,-5.480521
8,2023-07-10 08:00:00+00:00,8,38.954167,54.333865,-15.379698
9,2023-07-10 09:00:00+00:00,9,66.320834,48.872196,17.448638


---

In [52]:
# Extract the data for the selected date
daily_date = df_data[df_data['time'].dt.date == pd.to_datetime(selected_date).date()].reset_index(drop=True)
daily_date = daily_date[['time', 'hour', 'actual_demand', 'forecast_demand', 'error']]
if(len(daily_date)==24):
    print('The selected date has 24 hours')
else:
    print('The selected date does not have 24 hours')
daily_date


The selected date has 24 hours


,time,hour,actual_demand,forecast_demand,error
0,2023-07-10 00:00:00+00:00,0,19.491667,37.182262,-17.690595
1,2023-07-10 01:00:00+00:00,1,42.829167,44.471830,-1.642663
2,2023-07-10 02:00:00+00:00,2,144.383336,87.021062,57.362274
3,2023-07-10 03:00:00+00:00,3,42.000000,59.673830,-17.673829
4,2023-07-10 04:00:00+00:00,4,70.545834,63.712583,6.833251
5,2023-07-10 05:00:00+00:00,5,29.120834,61.861487,-32.740654
6,2023-07-10 06:00:00+00:00,6,45.058334,60.155867,-15.097533
7,2023-07-10 07:00:00+00:00,7,50.479167,55.959689,-5.480521
8,2023-07-10 08:00:00+00:00,8,38.954167,54.333865,-15.379698
9,2023-07-10 09:00:00+00:00,9,66.320834,48.872196,17.448638


---

## Methode: Monte Carlo Sampling and Roulette Wheel Mechanism


In [53]:
import numpy as np
import random
from scipy import stats

# Number of intervals
num_intervals = 7

# Function to randomly select an interval based on the CDF
def select_interval(cumulative_probabilities):
    rnd = random.random()
    for i, cp in enumerate(cumulative_probabilities):
        if rnd <= cp:
            return i
    return len(cumulative_probabilities) - 1

# Get the list of dates in the selected period
date_range = pd.date_range(start=start_date.normalize(), end=end_date.normalize(), freq='D', tz='UTC')

# Initialize dictionaries to store scenario data
scenario_values = {f"Scenario{idx+1}": [] for idx in range(num_scenarios)}
scenario_probs = {f"Scenario{idx+1}": 1 for idx in range(num_scenarios)}

# Generate scenarios for each day
for single_date in date_range:
    # Extract data for the current day
    day_data = period_data[period_data['time'].dt.normalize() == single_date].reset_index(drop=True)
    
    if len(day_data) != 24:
        print(f"Warning: {single_date.date()} does not have 24 hours of data. Skipping this day.")
        continue  # Skip this day or handle as needed
    
    # Generate scenarios for this day
    daily_scenarios = []
    daily_probabilities = []
    
    for scenario_index in range(num_scenarios):
        scenario_name = f"Scenario{scenario_index + 1}"
        scenario = []
        scenario_probability = 1.0  # Initialize the scenario probability
        
        for t in range(24):  # 24 hours in a day
            # Get the hour for the current time step
            hour = day_data['hour'].iloc[t]
            
            # Fetch mu and sigma for the current hour
            mu = error_dict[hour]['mu']
            sigma = error_dict[hour]['sigma']
            
            # Define the intervals
            intervals = np.linspace(mu - 3 * sigma, mu + 3 * sigma, num_intervals + 1)
            interval_centers = (intervals[:-1] + intervals[1:]) / 2
            
            # Calculate probabilities for each interval
            probabilities = [
                stats.norm.cdf(intervals[i + 1], mu, sigma) - stats.norm.cdf(intervals[i], mu, sigma)
                for i in range(len(intervals) - 1)
            ]
            cum_probabilities = np.cumsum(probabilities)
            
            # Select interval index based on cumulative probabilities
            interval_index = select_interval(cum_probabilities)
            
            # Calculate forecast error and add to forecast demand
            forecast_error = interval_centers[interval_index]
            actual_demand = day_data['forecast_demand'].iloc[t] + forecast_error
            scenario.append(actual_demand)
            
            # Multiply the probability of this interval to the scenario probability
            scenario_probability *= probabilities[interval_index]
        
        # Append the scenario values and probabilities for this day
        daily_scenarios.append(scenario)
        daily_probabilities.append(scenario_probability)
    
    # Normalize the daily probabilities
    total_daily_probability = sum(daily_probabilities)
    daily_probabilities = [p / total_daily_probability for p in daily_probabilities]
    
    # Combine daily scenarios into full-period scenarios
    for scenario_index in range(num_scenarios):
        scenario_name = f"Scenario{scenario_index + 1}"
        scenario_values[scenario_name].extend(daily_scenarios[scenario_index])
        scenario_probs[scenario_name] *= daily_probabilities[scenario_index]

# Normalize the scenario probabilities over the entire period
total_period_probability = sum(scenario_probs.values())
for scenario_name in scenario_probs:
    scenario_probs[scenario_name] /= total_period_probability

# Prepare the final data
heat_demand_data = {}
for scenario_name in scenario_values:
    heat_demand_data[scenario_name] = {
        "Probability": scenario_probs[scenario_name],
        "Values": scenario_values[scenario_name]
    }

# Output the scenarios and their normalized probabilities
for scenario_name, data in heat_demand_data.items():
    print(f"{scenario_name}:")
    print(f"Normalized Probability: {data['Probability']:.6f}")
    print(f"Values: {data['Values']}\n")


Scenario1:
Normalized Probability: 0.001970
Values: [3.0059580830236143, 35.754128355000724, 125.6491535079943, 48.454474199439275, 93.9958350110197, 32.461112647545164, -0.6882003322296342, 46.994503336706764, 50.07913464251907, 68.15090988055726, 28.879549595593666, 6.694166534909236, 104.93100591376506, 38.80337741488967, 63.83631291607238, 14.758572459242657, 50.68113703016716, 30.880808379384504, 61.48601511585353, 45.66209774941836, 15.293758070339184, 36.14163990094476, 16.560598437211524, 20.004605901535854]

Scenario2:
Normalized Probability: 0.000022
Values: [24.450603646356576, 57.857976293222926, 91.31011551898871, 79.2225668772724, 68.3745466358979, 6.2110827410737315, 23.901071178921917, 99.95158705279005, 75.01721845738552, 9.894680333629253, 28.879549595593666, 76.89844609137648, 5.0493121856952214, 61.99341038776518, 13.046019322551913, 71.64481172922575, 50.68113703016716, 3.050627322127852, 37.963387398211374, 45.66209774941836, 15.293758070339184, 19.09203824635906,

In [54]:
period_data

,time,hour,actual_demand,forecast_demand,error
0,2023-07-10 00:00:00+00:00,0,19.491667,37.182262,-17.690595
1,2023-07-10 01:00:00+00:00,1,42.829167,44.471830,-1.642663
2,2023-07-10 02:00:00+00:00,2,144.383336,87.021062,57.362274
3,2023-07-10 03:00:00+00:00,3,42.000000,59.673830,-17.673829
4,2023-07-10 04:00:00+00:00,4,70.545834,63.712583,6.833251
5,2023-07-10 05:00:00+00:00,5,29.120834,61.861487,-32.740654
6,2023-07-10 06:00:00+00:00,6,45.058334,60.155867,-15.097533
7,2023-07-10 07:00:00+00:00,7,50.479167,55.959689,-5.480521
8,2023-07-10 08:00:00+00:00,8,38.954167,54.333865,-15.379698
9,2023-07-10 09:00:00+00:00,9,66.320834,48.872196,17.448638


In [55]:
# Periodenbezeichnung definieren
period_str = {0: 'day', 1: 'week', 2: 'month'}

# Datumsstrings für den Dateinamen vorbereiten
start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')
filename_date = f"{start_date_str}_to_{end_date_str}_{period_str[selected_period]}"

# Erstellen des forecast_demand_dict mit der gewünschten Struktur
forecast_demand_dict = {
    "heat_demand": {
        str(t + 1): float(demand)
        for t, demand in enumerate(period_data['forecast_demand'])
    }
}

actual_demand_dict = {
    "heat_demand": {
        str(t + 1): float(demand)
        for t, demand in enumerate(period_data['actual_demand'])
    }
}


# Speichern des Forecast Demand in einer JSON-Datei
with open(f'{OUTPUT_PATH}heat_demand_{filename_date}.json', 'w') as json_file:
    json.dump(forecast_demand_dict, json_file, indent=4)

with open(f'{OUTPUT_PATH}actual_heat_demand_{filename_date}.json', 'w') as json_file:
    json.dump(actual_demand_dict, json_file, indent=4)


# Anpassen von heat_demand_data für die gewünschte Struktur
# heat_demand_scenarios = {}
# for scenario_name, data in heat_demand_data.items():
#     scenario_dict = {"Probability": data["Probability"]}
#     scenario_dict.update({
#         str(t + 1): float(value)
#         for t, value in enumerate(data["Values"])
#     })
#     heat_demand_scenarios[scenario_name] = scenario_dict

# Speichern der Szenarien in einer JSON-Datei
# with open(f'{OUTPUT_PATH}heat_demand_scenarios_{filename_date}.json', 'w') as json_file:
#     json.dump(heat_demand_scenarios, json_file, indent=4)

# # Laden der Daten bei Bedarf
# with open(f'{OUTPUT_PATH}heat_demand_{filename_date}.json', 'r') as json_file:
#     loaded_forecast_demand = json.load(json_file)

# with open(f'{OUTPUT_PATH}heat_demand_scenarios_{filename_date}.json', 'r') as json_file:
#     loaded_scenarios = json.load(json_file)


In [56]:
print(start_date)
print(end_date)

2023-07-10 00:00:00+00:00
2023-07-10 23:00:00+00:00


## Szenarien Reduktion

### Backward Scenario Reduction Methode

In [57]:
import numpy as np
import copy

# Copy the scenario data
S = copy.deepcopy(heat_demand_data)  # Current scenarios
DS = {}  # Deleted scenarios

# Extract the scenario names
scenario_names = list(S.keys())
num_scenarios = len(scenario_names)

# Extract the number of time periods from one of the scenarios
time_periods = range(len(S[scenario_names[0]]["Values"]))

# Precompute the distances between all pairs of scenarios
distance_matrix = np.zeros((num_scenarios, num_scenarios))

for i in range(num_scenarios):
    for j in range(i + 1, num_scenarios):
        s1 = S[scenario_names[i]]["Values"]
        s2 = S[scenario_names[j]]["Values"]
        # Compute the Euclidean distance
        distance = np.sqrt(sum((v1 - v2) ** 2 for v1, v2 in zip(s1, s2)))
        distance_matrix[i, j] = distance
        distance_matrix[j, i] = distance  # Symmetric matrix

# Perform the scenario reduction
while len(S) > desired_num_scenarios:
    # Step 2: For each scenario nk, determine nr with minimum distance DTk,r
    nearest_neighbor = {}
    for k_idx in range(num_scenarios):
        k_name = scenario_names[k_idx]
        if k_name not in S:
            continue
        distances = [
            (distance_matrix[k_idx, s_idx], scenario_names[s_idx])
            for s_idx in range(num_scenarios)
            if scenario_names[s_idx] in S and scenario_names[s_idx] != k_name
        ]
        if distances:
            min_distance, r_name = min(distances, key=lambda x: x[0])
            nearest_neighbor[k_name] = (min_distance, r_name)
    
    # Step 3: Calculate PDk,r = Pr(k) * DTk,r
    PD = {}
    for k_name in nearest_neighbor:
        Pr_k = S[k_name]["Probability"]
        DTk_r, r_name = nearest_neighbor[k_name]
        PD_k_r = Pr_k * DTk_r
        PD[k_name] = (PD_k_r, r_name)
    
    # Select d for which PDd = min PDk
    d_name, (min_PDd, r_name) = min(PD.items(), key=lambda x: x[1][0])
    
    # Step 4: Remove nd from S, add nd to DS, adjust probabilities
    Pr_d = S[d_name]["Probability"]
    Pr_r = S[r_name]["Probability"]
    S[r_name]["Probability"] = Pr_r + Pr_d
    DS[d_name] = S.pop(d_name)
    
    # Update the scenario names and number of scenarios
    scenario_names = list(S.keys())
    num_scenarios = len(scenario_names)

# Normalize the probabilities again to ensure they sum to 1
total_probability = sum(scenario["Probability"] for scenario in S.values())
for scenario in S.values():
    scenario["Probability"] /= total_probability

# Output the reduced scenarios and their probabilities
print("\nReduced Scenarios:")
for scenario_name, data in S.items():
    print(f"{scenario_name}:")
    print(f"  Normalized Probability: {data['Probability']:.4f}")
    print(f"  Values: {data['Values']}")



Reduced Scenarios:
Scenario21:
  Normalized Probability: 0.1575
  Values: [24.450603646356576, 13.650280416778529, 91.31011551898871, 48.454474199439275, 68.3745466358979, 58.7111425540166, 73.07961420122504, 46.994503336706764, 50.07913464251907, 39.02279510709326, 28.879549595593666, 53.497019572554066, 30.019735617712684, 15.613344442014153, 63.83631291607238, 71.64481172922575, 50.68113703016716, 58.71098943664116, 61.48601511585353, 25.18365860063245, 15.293758070339184, 2.0424365917733596, 16.560598437211524, -0.20916884735166263]
Scenario49:
  Normalized Probability: 0.0554
  Values: [24.450603646356576, 13.650280416778529, 56.97107752998314, 48.454474199439275, 68.3745466358979, 32.461112647545164, 23.901071178921917, 46.994503336706764, 50.07913464251907, 9.894680333629253, 28.879549595593666, 6.694166534909236, 5.0493121856952214, 38.80337741488967, 63.83631291607238, 43.2016920942342, 22.593722216093752, 58.71098943664116, 37.963387398211374, 66.14053689820426, 15.293758070

In [58]:
# Create a dictionary to store the reduced scenarios
reduced_heat_demand_scenarios = {}
for scenario_name, data in S.items():
    scenario_dict = {"Probability": data["Probability"]}
    scenario_dict.update({
        str(t + 1): float(value)
        for t, value in enumerate(data["Values"])
    })
    reduced_heat_demand_scenarios[scenario_name] = scenario_dict

# Save the reduced scenarios to a JSON file
with open(f'{OUTPUT_PATH}reduced_heat_demand_scenarios_{filename_date}.json', 'w') as json_file:
    json.dump(reduced_heat_demand_scenarios, json_file, indent=4)
